<a href="https://colab.research.google.com/github/qahtanaa/OnSubGroupFairness/blob/main/TESI_MAGISTRALE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install aif360

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.1/214.1 kB 2.0 MB/s eta 0:00:00


In [ ]:
##Download the files from these links and move these files to the path mentioned below
##https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
##https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
##
##
! mv /content/german.data /usr/local/lib/python3.6/dist-packages/aif360/datasets/../data/raw/german/german.data
! mv /content/german.doc  /usr/local/lib/python3.6/dist-packages/aif360/datasets/../data/raw/german/german.doc

mv: cannot stat '/content/german.data': No such file or directory
mv: cannot stat '/content/german.doc': No such file or directory


In [2]:
import numpy as np
import pandas as pd
from aif360.metrics import utils
from scipy.sparse import issparse

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


In [3]:
class DataPreparation():
  """
  ........
  """
  def __init__(self, df, sensitive, label, priv, unpriv, fav, unfav, categorical):
    """
    Construct all necessary attributes for the data preparation.

    df : (pandas DataFrame) containing the data
    sensitive : (list(str)) specifying the column names of all sensitive features
    label : (str) specifying the label column
    priv : (list(dicts)) representation of the privileged groups
    unpriv : (list(dicts)) representation of the unprivileged groups
    fav : (str/int/..) value representing the favorable label
    unfav : (str/int/..) value representing the unfavorable label
    categorical : (list(str)) (optional) specifying column names of categorical features
    """
    self.df = df
    self.sensitive = sensitive
    self.label = label
    self.priv = priv
    self.unpriv = unpriv
    self.fav = fav
    self.unfav = unfav
    self.categorical = categorical


  def detect_missing_values(self):
      """
      Detect rows with missing values and remove them from the DataFrame.
      """
      initial_rows = len(self.df)
      self.df = self.df.dropna()
      removed_rows = initial_rows - len(self.df)

      if removed_rows > 0:
          print(f"Detected {removed_rows} rows with missing values. Removed them.")
      else:
          print("No missing values detected.") #pass

  def binary_label(self):
    """
    Check that the decision label is made of two values, change it as a binary representation
    where favorable label = 1, unfavorable label = 0.
    """
    number_label_values = self.df[self.label].nunique()
    if number_label_values == 2:
      print(f"The '{self.label}' column has only two unique values.")
      self.df[self.label].replace([self.unfav, self.fav], [0, 1], inplace=True)
    else:
      print(f"The '{self.label}' column does not have exactly two unique values, as it should.")

  def find_categorical_attributes(self):
    """
    find the categorical attributes
    """
    attribute_types = {}

    for column in self.df.columns:
        # Skip the 'Group' column
        if column == 'Group':
            continue
        # if the column has already been classified as categorical by the user, leave it cat
        if column in self.categorical:
            attribute_types[column] = 'Categorical'
        # if the type of the column is np.number and is not binary, consider it as numerical
        elif np.issubdtype(self.df[column].dtype, np.number) and self.df[column].nunique() != 2:
            attribute_types[column] = 'Numerical'
        # in the other cases, consider it as categorical
        else:
            attribute_types[column] = 'Categorical'

    return attribute_types


  def create_group_column(self):
    """
    Create a 'Group' column in the DataFrame based on protected attributes, privileged/unprivileged conditions, and label. Then drop
    the protected attributes and label columns because those informations are already present in the Group column
    """
    group_combinations = pd.MultiIndex.from_product([self.df[sensitive].unique() for sensitive in self.sensitive] + [self.df[self.label].unique()], names=self.sensitive + [self.label])
    #print(group_combinations)
    print(list(enumerate(group_combinations)))
    # Create a mapping between group combinations and their corresponding numbers
    group_mapping = {group: idx for idx, group in enumerate(group_combinations)}
    # Apply the mapping to create a new column in the DataFrame
    self.df['Group'] = pd.MultiIndex.from_frame(self.df[self.sensitive + [self.label]]).map(group_mapping)
    #do not drop the columns containing sensitive attributes and label because they'll be
    #needed to compute the distance between samples
    #self.df.drop(columns=[self.label] + self.sensitive, inplace=True)

  def prepare(self):
    """
    Perform all preprocessing steps.
    """
    self.detect_missing_values()
    self.binary_label()
    self.find_categorical_attributes()
    self.create_group_column()
    #self.normalization()
    #self.make_numerical()
    return self

In [4]:
########### CODE TO TRY DATAPREPARATION CLASS WITH A SMALL DATAFAME

# Create a small DataFrame
np.random.seed(42)
n = 10
data = {
    'Gender': np.random.choice(['F', 'M'], size=n),
    'Race': np.random.choice(['B', 'W'], size=n),
    'NumKids': np.random.choice(range(5), size=n),
    'Height': np.random.uniform(150, 190, size=n),
    'ShirtColor': np.random.choice(['Red', 'Blue', 'Green'], size=n),
    'Age': np.random.randint(18, 65, size=n),
    'Accepted': np.random.choice([1, 2], size=n)
}

df = pd.DataFrame(data)

# Introduce missing values randomly
missing_fraction = 0.05  # Adjust this fraction based on the desired percentage of missing values

# Randomly select cells and set their values to NaN
mask = np.random.rand(*df.shape) < missing_fraction
df[mask] = np.nan
print(df)
for column in df.columns:
    print(f"{column}: {df[column].dtype}")


#count the number of columns in the dataframe
num_of_columns = len(df.columns)
print("Number of columns:", num_of_columns)

#use the DataPreparation class to preprocess the dataframe
data_prep = DataPreparation(df, ['Gender','Race'], 'Accepted', ['M','W'], ['F', 'B'], 2, 1, ['NumKids'])
data_prep.prepare()
# use this reset index to have indices from 0 to len(df)
data_prep.df = data_prep.df.reset_index(drop=True)
print(data_prep.df)



  Gender Race  NumKids      Height ShirtColor  Age  Accepted
0      F    B      4.0  167.277801        Red   31         1
1      M    B      1.0  161.649166        NaN   26         1
2      F    B      3.0  174.474116        Red   43         2
3      F    B      1.0  155.579754        NaN   19         1
4      F    W      3.0  161.685786      Green   37         2
5      M    B      4.0  164.654474        Red   45         2
6      F    W      0.0  168.242799        Red   64         2
7      F    W      NaN  181.407038        NaN   24         2
8      F    W      1.0  157.986951       Blue   61         2
9      M    B      4.0  170.569378        Red   25         2
Gender: object
Race: object
NumKids: float64
Height: float64
ShirtColor: object
Age: int64
Accepted: int64
Number of columns: 7
Detected 3 rows with missing values. Removed them.
The 'Accepted' column has only two unique values.
[(0, ('F', 'B', 0)), (1, ('F', 'B', 1)), (2, ('F', 'W', 0)), (3, ('F', 'W', 1)), (4, ('M', 'B', 0)),

<ipython-input-3-66d4d042f49e>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[self.label].replace([self.unfav, self.fav], [0, 1], inplace=True)
<ipython-input-3-66d4d042f49e>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['Group'] = pd.MultiIndex.from_frame(self.df[self.sensitive + [self.label]]).map(group_mapping)


In [5]:
# Call the method to find categorical indices
attribute_types = data_prep.find_categorical_attributes()

# Print the result
print("dictionary 'Attribute' : 'Numerical/Categorical' \n ", attribute_types)

# Initialize cat_features list
cat_features = []

# Iterate over attribute names and determine whether they are categorical or numerical
for attr in attribute_types:
    cat_features.append(attribute_types[attr] == 'Categorical')

print('boolean categorical features vector:\n', cat_features)


dictionary 'Attribute' : 'Numerical/Categorical' 
  {'Gender': 'Categorical', 'Race': 'Categorical', 'NumKids': 'Categorical', 'Height': 'Numerical', 'ShirtColor': 'Categorical', 'Age': 'Numerical', 'Accepted': 'Categorical'}
boolean categorical features vector:
 [True, True, True, False, True, False, True]


___________________________________GOWER DISTANCE, USING ITS PACKAGE

In [6]:
!pip install gower

In [7]:
import gower

# Dataframe after the DataPreparation, it does not have missing values ecc
# List of columns to exclude: 'Group'.
# To compute the distance consider all attributes (including sensitive and label)
columns_to_exclude = ['Group']

# Create a new DataFrame (Xd) by excluding specified columns
Xd = data_prep.df.drop(columns=columns_to_exclude, axis=1)
print(Xd)

#cat_features introduced above

# Consider the categorical attributes as categorical
matrix_dist_gower = gower.gower_matrix(Xd, cat_features = cat_features)
print('Matrix of distances using Gower\n', matrix_dist_gower)

# TOP N neighbors, general function:
#topn = gower.gower_topn(Xd.iloc[2:3,:], Xd.iloc[:,], cat_features = cat_features, n = 5)
#print(topn)

# Number of rows in Xd
num_rows_gower = Xd.shape[0]

# Initialize a list to store the results
matrix_neighbors_gower = []

# Loop through each row
for i in range(num_rows_gower):
  top_n_neighbors_gower = gower.gower_topn(Xd.iloc[i:i+1, :], Xd.iloc[:, :], cat_features = cat_features, n=5)
  matrix_neighbors_gower.append(top_n_neighbors_gower)

# Convert the list of dictionaries to a matrix
matrix_neighbors_gower_index = np.array([item['index'] for item in matrix_neighbors_gower])
matrix_neighbors_gower_values = np.array([item['values'] for item in matrix_neighbors_gower])

# Print or use the resulting matrix as needed
print("Matrix of Indices, using Gower distance:")
print(matrix_neighbors_gower_index)
print("\nMatrix of Values, using Gower distance:")
print(matrix_neighbors_gower_values)


  Gender Race  NumKids      Height ShirtColor  Age  Accepted
0      F    B      4.0  167.277801        Red   31         0
1      F    B      3.0  174.474116        Red   43         1
2      F    W      3.0  161.685786      Green   37         1
3      M    B      4.0  164.654474        Red   45         1
4      F    W      0.0  168.242799        Red   64         1
5      F    W      1.0  157.986951       Blue   61         1
6      M    B      4.0  170.569378        Red   25         1
Matrix of distances using Gower
 [[0.         0.39202458 0.64186    0.3597268  0.55781204 0.7618215
  0.336213  ]
 [0.39202458 0.         0.41849995 0.37812498 0.41663015 0.6373626
  0.38548192]
 [0.64186    0.41849995 0.         0.62645555 0.44143024 0.40567583
  0.6923587 ]
 [0.3597268  0.37812498 0.62645555 0.         0.52926046 0.687809
  0.12451121]
 [0.55781204 0.41663015 0.44143024 0.52926046 0.         0.3855676
  0.5915878 ]
 [0.7618215  0.6373626  0.40567583 0.687809   0.3855676  0.
  0.81232023]


_______HEOM DISTANCE - try to implement


In [8]:
# Implement HEOM distance (as in Wilson's 1997 paper), starting from the source code of Gower
# and modifying it where needed (https://github.com/wwwjk366/gower/blob/master/gower/gower_dist.py)

def HEOM_matrix(data_x, data_y=None, cat_features=None):
    # function checks
    X = data_x
    if data_y is None: Y = data_x
    else: Y = data_y
    if not isinstance(X, np.ndarray):
        if not np.array_equal(X.columns, Y.columns): raise TypeError("X and Y must have same columns!")
    else:
         if not X.shape[1] == Y.shape[1]: raise TypeError("X and Y must have same y-dim!")

    if issparse(X) or issparse(Y): raise TypeError("Sparse matrices are not supported!")

    x_n_rows, x_n_cols = X.shape
    y_n_rows, y_n_cols = Y.shape

    if cat_features is None:
        if not isinstance(X, np.ndarray):
            is_number = np.vectorize(lambda x: not np.issubdtype(x, np.number))
            cat_features = is_number(X.dtypes)
        else:
            cat_features = np.zeros(x_n_cols, dtype=bool)
            for col in range(x_n_cols):
                if not np.issubdtype(type(X[0, col]), np.number):
                    cat_features[col]=True
    else:
        cat_features = np.array(cat_features)
    # print(cat_features)

    if not isinstance(X, np.ndarray): X = np.asarray(X)
    if not isinstance(Y, np.ndarray): Y = np.asarray(Y)

    Z = np.concatenate((X,Y))
    x_index = range(0,x_n_rows)
    y_index = range(x_n_rows,x_n_rows+y_n_rows)
    Z_num = Z[:,np.logical_not(cat_features)] # colonne con numerical attributes

    num_cols = Z_num.shape[1] #numero di colonne in Z con  numer attr, in quest caso due, cioè Height e Age
    num_ranges = np.zeros(num_cols) #list or 2 elements to save the range for Height and the range for Age
    num_max = np.zeros(num_cols) #list or 2 elements to save the max for Height and the range for Age
    num_min = np.zeros(num_cols) #list or 2 elements to save the min for Height and the range for Age

    for col in range(num_cols):
      col_array = Z_num[:, col].astype(np.float32)
      max = np.nanmax(col_array)
      min = np.nanmin(col_array)
      if np.isnan(max):
        max = 0.0
      if np.isnan(min):
        min = 0.0
      num_max[col] = max
      num_min[col] = min
      num_ranges[col] = max - min if (max != 0) else 0.0
      # quindi num_max, num_min, num_ranges sono liste

    Z_cat = Z[:,cat_features]

    X_cat = Z_cat[x_index,]
    X_num = Z_num[x_index,]
    Y_cat = Z_cat[y_index,]
    Y_num = Z_num[y_index,]
    #print('all matrix', X_cat,X_num,Y_cat,Y_num)
    out = np.zeros((x_n_rows, y_n_rows), dtype=np.float32)
    for i in range(x_n_rows):
      if x_n_rows == y_n_rows:
        for j in range(i, y_n_rows):
          dist = HEOM_get(X_cat[i,:], X_num[i,:],
                   Y_cat[j,:], Y_num[j,:],
                   cat_features, num_ranges)
          out[i,j]=dist
          out[j,i]=dist
      else:
        for j in range(y_n_rows):
          dist = HEOM_get(X_cat[i,:], X_num[i,:],
                   Y_cat[j,:], Y_num[j,:],
                   cat_features, num_ranges)
          out[i,j]=dist

    return out



def HEOM_get(xi_cat,xi_num,xj_cat,xj_num,categorical_features,
              ranges_of_numeric):

  # categorical columns
    overlap_cat = np.where(xi_cat == xj_cat,np.zeros_like(xi_cat),np.ones_like(xi_cat))
    sum_squared_cat_overlap = np.sum(overlap_cat ** 2)

    # numerical columns
    rn_diff_numerator = np.absolute(xi_num-xj_num)
    rn_diff_denominator = ranges_of_numeric
    rn_diff = np.divide(rn_diff_numerator, rn_diff_denominator, out=np.zeros_like(rn_diff_numerator), where=ranges_of_numeric!=0)
    sum_squared_rn_diff = np.sum(rn_diff ** 2)

    # total sum of d^2, like in Wilson paper's
    sums= np.add(sum_squared_cat_overlap, sum_squared_rn_diff)
    d = np.sqrt(sums)
    return d

def smallest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    #n += 1
    flat = np.nan_to_num(ary.flatten(), nan=999)
    indices = np.argpartition(-flat, -n)[-n:]
    indices = indices[np.argsort(flat[indices])]
    #indices = np.delete(indices,0,0)
    values = flat[indices]
    return {'index': indices, 'values': values}

def HEOM_topn(data_x, data_y=None, cat_features=None, n = 5):

    if data_x.shape[0] >= 2: TypeError("Only support `data_x` of 1 row. ")
    dm = HEOM_matrix(data_x, data_y, cat_features)

    return smallest_indices(np.nan_to_num(dm[0], nan=1),n)

In [9]:
columns_to_exclude = ['Group']
X = data_prep.df.drop(columns=columns_to_exclude, axis=1)
print('dataframe\n', X)

matrix_dist_heom = HEOM_matrix(X, cat_features = cat_features)
print('Matrix of distances using HEOM\n', matrix_dist_heom)

#topn_heom = HEOM_topn(X, cat_features = cat_features)
# Number of rows in Xd
num_rows_heom = X.shape[0]

# Initialize a list to store the results
matrix_neighbors_heom = []
# Loop through each row
for i in range(num_rows_heom):
  top_n_heom = HEOM_topn(X.iloc[i:i+1, :], X.iloc[:, :], cat_features = cat_features, n=5)
  matrix_neighbors_heom.append(top_n_heom)

# Convert the list of dictionaries to a matrix
matrix_neighbors_index_heom = np.array([item['index'] for item in matrix_neighbors_heom])
matrix_neighbors_values_heom = np.array([item['values'] for item in matrix_neighbors_heom])

# Print or use the resulting matrix as needed
print("Matrix of Indices, using HEOM distance:")
print(matrix_neighbors_index_heom)
print("\nMatrix of Values, using HEOM distance:")
print(matrix_neighbors_values_heom)

dataframe
   Gender Race  NumKids      Height ShirtColor  Age  Accepted
0      F    B      4.0  167.277801        Red   31         0
1      F    B      3.0  174.474116        Red   43         1
2      F    W      3.0  161.685786      Green   37         1
3      M    B      4.0  164.654474        Red   45         1
4      F    W      0.0  168.242799        Red   64         1
5      F    W      1.0  157.986951       Blue   61         1
6      M    B      4.0  170.569378        Red   25         1
Matrix of distances using HEOM
 [[0.        1.5116842 2.0343814 1.4677124 1.9285752 2.2156875 1.436498 ]
 [1.5116842 0.        1.6202799 1.53537   1.5597392 2.0525637 1.506356 ]
 [2.0343814 1.6202799 0.        2.018539  1.6240253 1.5585344 2.0940392]
 [1.4677124 1.53537   2.018539  0.        1.8123776 2.081311  0.6258532]
 [1.9285752 1.5597392 1.6240253 1.8123776 0.        1.5468885 2.0049722]
 [2.2156875 2.0525637 1.5585344 2.081311  1.5468885 0.        2.3311996]
 [1.436498  1.506356  2.0940392

________HVDM DISTANCE - try to implement

In [10]:
# Implement HVDM distance (as in Wilson's 1997 paper), starting from the source code of Gower
# and modifying it where needed (https://github.com/wwwjk366/gower/blob/master/gower/gower_dist.py)

# DON'T USE THE COLUMN 'ACCEPTED' (THE LABEL) AS A CATEGORICAL ATTRIBUTE

def HVDM_matrix(data_x, target_class, data_y=None, cat_features=None):
    # function checks
    X = data_x #the dataframe without Label and Group columns
    target = target_class #column with labels
    if data_y is None: Y = data_x
    else: Y = data_y
    if not isinstance(X, np.ndarray):
        if not np.array_equal(X.columns, Y.columns): raise TypeError("X and Y must have same columns!")
    else:
         if not X.shape[1] == Y.shape[1]: raise TypeError("X and Y must have same y-dim!")

    if issparse(X) or issparse(Y): raise TypeError("Sparse matrices are not supported!")

    x_n_rows, x_n_cols = X.shape
    y_n_rows, y_n_cols = Y.shape

    if cat_features is None:
        if not isinstance(X, np.ndarray):
            is_number = np.vectorize(lambda x: not np.issubdtype(x, np.number))
            cat_features = is_number(X.dtypes)
        else:
            cat_features = np.zeros(x_n_cols, dtype=bool)
            for col in range(x_n_cols):
                if not np.issubdtype(type(X[0, col]), np.number):
                    cat_features[col]=True
    else:
        cat_features = np.array(cat_features)
    # print(cat_features)

    if not isinstance(X, np.ndarray): X = np.asarray(X)
    if not isinstance(Y, np.ndarray): Y = np.asarray(Y)

    Z = np.concatenate((X,Y))
    x_index = range(0,x_n_rows)
    y_index = range(x_n_rows,x_n_rows+y_n_rows)
    Z_num = Z[:,np.logical_not(cat_features)] # colonne con numerical attributes

    num_cols = Z_num.shape[1] #numero di colonne in Z con  numer attr, in quest caso due, cioè Height e Age
    num_std_dev = np.zeros(num_cols) #list or 2 elements to save the min for Height and the range for Age

    for col in range(num_cols):
        col_array = Z_num[:, col].astype(np.float32)
        std_dev = np.nanstd(col_array)
        if np.isnan(std_dev):
            std_dev = 0.0
        num_std_dev[col] = 4*std_dev #list containing 4*std for every column


    Z_cat = Z[:,cat_features]

    X_cat = Z_cat[x_index,]
    X_num = Z_num[x_index,]
    Y_cat = Z_cat[y_index,]
    Y_num = Z_num[y_index,]

    # matrix with categorical attributes and label that will be used in VDM
    X_cat_label = np.column_stack((X_cat, target))
    ####VDM to compute all the values, most likely will return a dictionary
    nax_naxc_values = VDM(X_cat_label)

    #print('all matrix', X_cat,X_num,Y_cat,Y_num)
    out = np.zeros((x_n_rows, y_n_rows), dtype=np.float32)
    for i in range(x_n_rows):
      for j in range(y_n_rows):
        dist = HVDM_get(X_cat[i,:], X_num[i,:],
                   Y_cat[j,:], Y_num[j,:], nax_naxc_values,
                   cat_features, num_std_dev)
        out[i,j]=dist
    #
    #for i in range(x_n_rows):
    #  if x_n_rows == y_n_rows:
    #    for j in range(i, y_n_rows):
    #      dist = HEOM_get(X_cat[i,:], X_num[i,:],
    #               Y_cat[j,:], Y_num[j,:],
    #               cat_features, num_ranges)
    #      out[i,j]=dist
    #      out[j,i]=dist
    #  else:
    #    for j in range(y_n_rows):
    #      dist = HEOM_get(X_cat[i,:], X_num[i,:],
    #               Y_cat[j,:], Y_num[j,:],
    #               cat_features, num_ranges)
    #      out[i,j]=dist

    return out



def HVDM_get(xi_cat,xi_num,xj_cat,xj_num,nax_naxc_values,categorical_features,
              std_of_numeric):

    # categorical columns I NEED VDM FUNCTION
    results = nax_naxc_values
    dist_cat = np.zeros(len(xi_cat))
    for col in range(len(xi_cat)):
      Nax = results[col]['Nax'][xi_cat[col]]
      Nay = results[col]['Nax'][xj_cat[col]]
      Naxc_1 = results[col]['Naxc'][1][xi_cat[col]]
      Nayc_1 = results[col]['Naxc'][1][xj_cat[col]]
      Naxc_0 = results[col]['Naxc'][0][xi_cat[col]]
      Nayc_0 = results[col]['Naxc'][0][xj_cat[col]]
      dist_cat[col] = abs(np.divide(Naxc_0, Nax) - np.divide(Nayc_0, Nay))**2 +\
                      abs(np.divide(Naxc_1, Nax) - np.divide(Nayc_1, Nay))**2
    sum_cat = np.sum(dist_cat)

    # numerical columns
    normalized_diff_numerator = np.absolute(xi_num-xj_num)
    normalized_diff_denominator = std_of_numeric
    normalized_diff = np.divide(normalized_diff_numerator, normalized_diff_denominator, out=np.zeros_like(normalized_diff_numerator), where=std_of_numeric!=0)
    sum_squared_normalized_diff = np.sum(normalized_diff ** 2)

    # total sum of d^2, like in Wilson paper's
    sums= np.add(sum_cat, sum_squared_normalized_diff)
    d = np.sqrt(sums)
    return d

def VDM(data_x):
    """
    This function computes all the values needed for the normalized_vdm presented in
    Wilson's paper. The It returns a dictionary with all the values (so we don't have to
    compute them every time)
    """
    X_cat_label = data_x
    results = {}
    #loop for to check every column
    #range(matrix-1) so we don't consider the target column as categorical
    for column_idx in range(X_cat_label.shape[1]-1):
      column_values = X_cat_label[:, column_idx]
      unique_values= np.unique(column_values)

      # Ensure all unique values are represented and counted, even if the count is zero
      all_counts = np.zeros_like(unique_values, dtype=int)
      for i, value in enumerate(unique_values):
        all_counts[i] = np.sum(column_values == value)

      # Create dictionary for Nax
      nax_dict = dict(zip(unique_values, all_counts))

      # now we have to compute the Naxc values, that take into account
      # the target label
      column_and_label = np.column_stack((column_values, target))

      # Filter the matrix based on the target value (0/1)
      filtered_matrix_1 = column_and_label[column_and_label[:, 1] == 1]
      filtered_matrix_0 = column_and_label[column_and_label[:, 1] == 0]

      # Get unique values and their counts after filtering
      values_filt_1 = np.unique(filtered_matrix_1[:, 0])
      all_counts_1 = np.zeros_like(unique_values, dtype=int)
      for i, value in enumerate(unique_values):
        all_counts_1[i] = np.sum(filtered_matrix_1[:,0] == value)

      values_filt_0 = np.unique(filtered_matrix_0[:, 0])
      all_counts_0 = np.zeros_like(unique_values, dtype=int)
      for i, value in enumerate(unique_values):
        all_counts_0[i] = np.sum(filtered_matrix_0[:,0] == value)

      # Create dictionary for Naxc
      naxc_dict_1 = dict(zip(unique_values, all_counts_1))
      naxc_dict_0 = dict(zip(unique_values, all_counts_0))
      naxc_dict = dict(zip([1,0], [naxc_dict_1,naxc_dict_0]))

      # Assign dictionaries to results
      results[column_idx] = {'Nax': nax_dict, 'Naxc': naxc_dict}

    return results


def smallest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    #n += 1
    flat = np.nan_to_num(ary.flatten(), nan=999)
    indices = np.argpartition(-flat, -n)[-n:]
    indices = indices[np.argsort(flat[indices])]
    #indices = np.delete(indices,0,0)
    values = flat[indices]
    return {'index': indices, 'values': values}

def HVDM_topn(data_x,  target_class, data_y=None, cat_features=None, n = 5):
    target = target_class
    #if data_x.shape[0] >= 2: TypeError("Only support `data_x` of 1 row. ")
    dm = HVDM_matrix(data_x, target, data_y, cat_features)
    num_rows = dm.shape[0]
    smallest_indices_dict = []
    for i in range(num_rows):
      smallest_indices_dict.append(smallest_indices(np.nan_to_num(dm[i], nan=1),n))
    return smallest_indices_dict



In [11]:
X = data_prep.df.drop(columns=['Group', 'Accepted'], axis=1)

# Initialize the new cat_features list, since I am not considering the Accepted attribute
new_cat_features = []
# Iterate over the columns of the DataFrame
for column in X.columns:
    # Check if the column is categorical based on the attribute_types dictionary
    if attribute_types.get(column) == 'Categorical':
        new_cat_features.append(True)
    else:
        new_cat_features.append(False)

target = data_prep.df['Accepted'].values
target = target.reshape(-1,1)

matrix_dist_hvdm = HVDM_matrix(X, target, cat_features=new_cat_features)
print(matrix_dist_hvdm)

# Number of rows in Xd
num_rows_hvdm = X.shape[0]

# Initialize a list to store the results
matrix_neighbors_hvdm = []
print(isinstance(X, np.ndarray))
# Loop through each row

matrix_neighbors_hvdm= HVDM_topn(X, target, cat_features = new_cat_features, n=5)
#matrix_neighbors_hvdm.append(top_n_hvdm)

# Convert the list of dictionaries to a matrix
matrix_neighbors_index_hvdm = np.array([item['index'] for item in matrix_neighbors_hvdm])
matrix_neighbors_values_hvdm = np.array([item['values'] for item in matrix_neighbors_hvdm])

# Print or use the resulting matrix as needed
print("Matrix of Indices, using HVDM distance:")
print(matrix_neighbors_index_hvdm)
print("\nMatrix of Values, using HVDM distance:")
print(matrix_neighbors_values_hvdm)

[[0.         0.6291267  0.7172685  0.40499562 0.85133284 0.9715985
  0.3440043 ]
 [0.6291267  0.         0.7802792  0.73102003 0.6082136  0.98349804
  0.67110324]
 [0.7172685  0.7802792  0.         0.74187136 0.6589006  0.48089525
  0.8635611 ]
 [0.40499562 0.73102003 0.74187136 0.         0.76318854 0.8377672
  0.4707386 ]
 [0.85133284 0.6082136  0.6589006  0.76318854 0.         0.578663
  0.98202866]
 [0.9715985  0.98349804 0.48089525 0.8377672  0.578663   0.
  1.1544718 ]
 [0.3440043  0.67110324 0.8635611  0.4707386  0.98202866 1.1544718
  0.        ]]
False
Matrix of Indices, using HVDM distance:
[[0 6 3 1 2]
 [1 4 0 6 3]
 [2 5 4 0 3]
 [3 0 6 1 2]
 [4 5 1 2 3]
 [5 2 4 3 0]
 [6 0 3 1 2]]

Matrix of Values, using HVDM distance:
[[0.         0.3440043  0.40499562 0.6291267  0.7172685 ]
 [0.         0.6082136  0.6291267  0.67110324 0.73102003]
 [0.         0.48089525 0.6589006  0.7172685  0.74187136]
 [0.         0.40499562 0.4707386  0.73102003 0.74187136]
 [0.         0.578663   0.60

compare these distances

In [13]:
print('dataframe\n', X)
print("Matrix of Indices, using Gower distance:")
print(matrix_neighbors_gower_index)
print('\n')
print("Matrix of Indices, using HEOM distance:")
print(matrix_neighbors_index_heom)
print('\n')
print("Matrix of Indices, using HVDM distance:")
print(matrix_neighbors_index_hvdm)

dataframe
   Gender Race  NumKids      Height ShirtColor  Age
0      F    B      4.0  167.277801        Red   31
1      F    B      3.0  174.474116        Red   43
2      F    W      3.0  161.685786      Green   37
3      M    B      4.0  164.654474        Red   45
4      F    W      0.0  168.242799        Red   64
5      F    W      1.0  157.986951       Blue   61
6      M    B      4.0  170.569378        Red   25
Matrix of Indices, using Gower distance:
[[0 6 3 1 4]
 [1 3 6 0 4]
 [2 5 1 4 3]
 [3 6 0 1 4]
 [4 5 1 2 3]
 [5 4 2 1 3]
 [6 3 0 1 4]]


Matrix of Indices, using HEOM distance:
[[0 6 3 1 4]
 [1 6 0 3 4]
 [2 5 1 4 3]
 [3 6 0 1 4]
 [4 5 1 2 3]
 [5 4 2 1 3]
 [6 3 0 1 4]]


Matrix of Indices, using HVDM distance:
[[0 6 3 1 2]
 [1 4 0 6 3]
 [2 5 4 0 3]
 [3 0 6 1 2]
 [4 5 1 2 3]
 [5 2 4 3 0]
 [6 0 3 1 2]]


In [15]:
X

,Gender,Race,NumKids,Height,ShirtColor,Age
0,F,B,4.0,167.277801,Red,31
1,F,B,3.0,174.474116,Red,43
2,F,W,3.0,161.685786,Green,37
3,M,B,4.0,164.654474,Red,45
4,F,W,0.0,168.242799,Red,64
5,F,W,1.0,157.986951,Blue,61
6,M,B,4.0,170.569378,Red,25
